In [1]:
import os
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

/Users/timchristy/.local/share/virtualenvs/agriculture-price-analysis-wczIgwxP/lib/python3.8/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/timchristy/.local/share/virtualenvs/agriculture-price-analysis-wczIgwxP/lib/python3.8/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
path_to_data = '../../data/00_raw/agriculture_prices.db'
conn = sqlite3.connect(path_to_data)
c = conn.cursor()

In [3]:
produce_df = pd.DataFrame(c.execute('''SELECT * FROM agriculture_prices''').fetchall())

In [4]:
produce_df.columns = ['Farm Price', 'Atlanta Retail', 'Chicago Retail', 'Los Angeles Retail', 'NYC Retail', 'Avg Spread', 'Commodity', 'Date']

In [5]:
produce_df.head()

,Farm Price,Atlanta Retail,Chicago Retail,Los Angeles Retail,NYC Retail,Avg Spread,Commodity,Date
0,1.16,2.23,1.70,1.99,2.54,82.33%,Strawberries,2019-05-19
1,0.91,2.67,1.89,2.47,2.66,166.21%,Strawberries,2019-05-12
2,0.58,2.42,1.89,2.12,2.91,302.59%,Strawberries,2019-05-05
3,0.58,2.79,1.95,2.38,2.96,334.48%,Strawberries,2019-04-28
4,0.69,2.92,2.04,2.38,3.16,280.43%,Strawberries,2019-04-21


In [6]:
produce_df.tail()

,Farm Price,Atlanta Retail,Chicago Retail,Los Angeles Retail,NYC Retail,Avg Spread,Commodity,Date
213791,0.28,1.69,0.0,1.24,1.19,267.86%,Nectarines,2000-07-23
213792,0.26,1.61,0.0,1.37,1.19,300.96%,Nectarines,2000-07-16
213793,0.30,1.71,0.0,1.42,1.19,260.00%,Nectarines,2000-07-09
213794,0.33,1.96,0.0,0.92,1.04,196.97%,Nectarines,2000-07-02
213795,0.39,1.22,0.0,0.92,1.19,113.46%,Nectarines,2000-06-25


In [7]:
produce_df.index = pd.to_datetime(produce_df['Date'])

In [8]:
conn.close()

# Cleaning Data
Want to update all prices to reflect 2019 dollars (adjust for inflation), and will be taking an average price for all retail prices. A dataframe with mean annual farm prices, mean annual average retail, farm prices standard deviation, average retail standard deviation, and year as an index will be created.

Since this needs to be done by commodity, I am also going to make a dictionary that holds data frames by commodity, produce_dict. So for example, typing ``produce_dict['Strawberries']`` will return a dataframe concerning only strawberries.

In [14]:
produce_list = list(produce_df['Commodity'].unique())
produce_list

['Strawberries',
 'Romaine Lettuce',
 'Red Leaf Lettuce',
 'Potatoes',
 'Oranges',
 'Iceberg Lettuce',
 'Green Leaf Lettuce',
 'Celery',
 'Cauliflower',
 'Carrots',
 'Cantaloupe',
 'Broccoli Crowns',
 'Avocados',
 'Broccoli Bunches',
 'Asparagus',
 'Flame Grapes',
 'Thompson Grapes',
 'Honeydews',
 'Tomatoes',
 'Plums',
 'Peaches',
 'Nectarines']

In [15]:
produce_dict = dict()
for produce in produce_list:
    produce_dict.setdefault(produce, produce_df[produce_df['Commodity'] == produce])

# Creating Average Retail Prices 
Averaging LA, NYC, etc. retail prices into one. Also making a standard deviation column for this.

In [31]:
produce_dict['Strawberries'].head()

,Farm Price,Atlanta Retail,Chicago Retail,Los Angeles Retail,NYC Retail,Avg Spread,Commodity,Date
Date,,,,,,,,
2019-05-19,1.16,2.23,1.70,1.99,2.54,82.33%,Strawberries,2019-05-19
2019-05-12,0.91,2.67,1.89,2.47,2.66,166.21%,Strawberries,2019-05-12
2019-05-05,0.58,2.42,1.89,2.12,2.91,302.59%,Strawberries,2019-05-05
2019-04-28,0.58,2.79,1.95,2.38,2.96,334.48%,Strawberries,2019-04-28
2019-04-21,0.69,2.92,2.04,2.38,3.16,280.43%,Strawberries,2019-04-21


In [33]:
for item in produce_list:
    avg_retail = [np.mean(x[1:5]) for x in produce_dict[item].values]
    avg_retail_std = [np.std(x[1:5]) for x in produce_dict[item].values]
    produce_dict[item]['avg_retail'] = avg_retail
    produce_dict[item]['avg_retail_std'] = avg_retail_std
    

<ipython-input-33-e8d73af7f0dd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produce_dict[item]['avg_retail'] = avg_retail
<ipython-input-33-e8d73af7f0dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produce_dict[item]['avg_retail_std'] = avg_retail_std
